In [ ]:
import tensorflow as tf
import numpy as np
from config import Config  # Assume config.py exists with all settings
from data.data_loader import DataLoader
from models.vgg16 import VGG16
from models.model_utils import ModelUtils
from utils.overfitting_monitor import OverfittingMonitor, AdaptiveRegularization
from visualization import Visualizer
from utils.logger import Logger

In [ ]:
def main():
    # Initialize config (adjust as needed)
    config = Config()  # Load or define config with batch_size, epochs, etc.
    
    # Initialize logger
    logger = Logger(config)
    logger.info("Starting Uncertainty Quantification in Explainable Vision Models")
    
    # Load data (using CIFAR-10 as example; can switch to others)
    data_loader = DataLoader(config)
    train_ds, val_ds, test_ds = data_loader.load_cifar10()
    logger.info("Data loaded successfully")
    
    # Build model
    vgg = VGG16(config)
    model = vgg.build_cifar10_model(num_classes=config.num_classes_cifar10, 
                                    input_shape=config.input_shape_cifar10)
    logger.info("Model built")
    
    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    # Initialize overfitting monitor and adaptive regularization
    monitor = OverfittingMonitor(patience=5, threshold=0.05)
    adaptive_reg = AdaptiveRegularization(initial_dropout=config.dropout_rate)
    
    # Training loop with callbacks
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
    for epoch in range(config.epochs):
        # Train one epoch
        train_loss, train_acc = model.evaluate(train_ds, verbose=0)  # Simplified; use fit for real training
        val_loss, val_acc = model.evaluate(val_ds, verbose=0)
        
        # Compute average UQ on val (using new method)
        val_images = next(iter(val_ds))[0]  # Get a batch
        _, avg_uq = vgg.predict_with_uq(model, val_images)
        avg_uq = np.mean(avg_uq)
        
        # Update monitor (with UQ)
        overfitting_detected = monitor.update(train_acc, val_acc, avg_uq)
        
        # Adjust regularization if overfitting
        score = monitor.get_overfitting_score()
        new_dropout = adaptive_reg.adjust(score)
        adaptive_reg.update_model_dropout(model, new_dropout)
        
        # Log and save history
        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)
        logger.info(f"Epoch {epoch+1}: Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}, Avg UQ {avg_uq:.6f}")
        
        if overfitting_detected:
            logger.warning("Overfitting detected; stopping early")
            break
    
    # Model analysis using ModelUtils
    params = ModelUtils.count_parameters(model)
    flops = ModelUtils.compute_flops(model)
    logger.info(f"Model Params: {params['total']}, FLOPs: {flops}")
    
    # Evaluation on test with UQ and XAI
    test_images, test_labels = next(iter(test_ds))
    mean_preds, uncertainty = vgg.predict_with_uq(model, test_images)
    pred_classes = np.argmax(mean_preds, axis=1)
    
    # XAI: Grad-CAM on first sample
    gradcam = ModelUtils.GradCAM(model, 'conv5_block3_conv3')  # Example layer
    heatmap = gradcam.compute_heatmap(test_images[0:1], pred_classes[0])
    
    # Visualize
    visualizer = Visualizer(config)
    visualizer.plot_training_history(history)
    visualizer.plot_gradcam(test_images[0].numpy(), heatmap, class_name="Predicted Class")
    visualizer.plot_uncertainty_map(uncertainty)
    
    logger.info("Training and evaluation completed")

In [ ]:
if __name__ == "__main__":
    main()